In [1]:
import os
import tqdm
import numpy as np
import pandas as pd

In [2]:
# Data loader for .mot files

def read_header(mot_file): 
    if not os.path.isfile(mot_file): 
        return []

    try:         
        with open(mot_file, 'r') as f:
            lines = f.readlines()
            headers = lines[10]
            headers = headers.split()
            return headers
    except Exception as e:
        print(f"Unable to load headers for file:{mot_file}. Error:{e}")
        return [] 

def storage_to_numpy(storage_file, excess_header_entries=0):
    """Returns the data from a storage file in a numpy format. Skips all lines
    up to and including the line that says 'endheader'.
    Parameters
    ----------
    storage_file : str
        Path to an OpenSim Storage (.sto) file.
    Returns
    -------
    data : np.ndarray (or numpy structure array or something?)
        Contains all columns from the storage file, indexable by column name.
    excess_header_entries : int, optional
        If the header row has more names in it than there are data columns.
        We'll ignore this many header row entries from the end of the header
        row. This argument allows for a hacky fix to an issue that arises from
        Static Optimization '.sto' outputs.
    Examples
    --------
    Columns from the storage file can be obtained as follows:
        >>> data = storage2numpy('<filename>')
        >>> data['ground_force_vy']
    """
    # What's the line number of the line containing 'endheader'?
    f = open(storage_file, 'r')

    header_line = False
    for i, line in enumerate(f):
        if header_line:
            column_names = line.split()
            break
        if line.count('endheader') != 0:
            line_number_of_line_containing_endheader = i + 1
            header_line = True
    f.close()
    # With this information, go get the data.
    if excess_header_entries == 0:
        names = True
        skip_header = line_number_of_line_containing_endheader
    else:
        names = column_names[:-excess_header_entries]
        skip_header = line_number_of_line_containing_endheader + 1
    data = np.genfromtxt(storage_file, names=names,
            skip_header=skip_header)

    new_data = []
    for d in data:
        new_data.append(list(d))
    new_data = np.array(new_data)

    return data


def storage_to_dataframe(storage_file, headers):
    # Extract data
    data = storage_to_numpy(storage_file)
    data = np.array(data)
    new_data = []
    for d in data:
        new_data.append(list(d))
    new_data = np.array(new_data)
    header_mapping = {header:i for i,header in enumerate(headers)}
    out = pd.DataFrame(data=data['time'], columns=['time'])
    for count, header in enumerate(headers):
        out.insert(count + 1, header, new_data[:,count+1])    
    
    return out

In [8]:
base_dir='/home/ubuntu/data/opencap-processing/Data/'

def load_kinematics_data(mcs_sessions): 
    simulation_data = {}

    for session_id in tqdm.tqdm(mcs_sessions): 
        simulation_results_path = os.path.join(base_dir, session_id, 'OpenSimData','Dynamics')
        for trial in os.listdir(simulation_results_path): 
            
            if trial == "SQT01": # Contains same results as segment-1 
                continue

            mot_file = os.path.join(simulation_results_path, trial, f'kinematics_activations_{trial}_torque_driven.mot')

            if not os.path.isfile(mot_file): 
                print(f"Simulation did not converge: ", session_id,trial)
                continue 
            mot_headers = read_header(mot_file)
            
            if len(mot_headers) == 0:
                print(f"Unable to load headers for file:", session_id,trial)
                continue 
            
            mot_headers = mot_headers
            
            # Remove time from headers
            mot_headers.remove('time')
            
            mot_data = data = storage_to_dataframe(mot_file, headers = mot_headers)
            
            if session_id not in simulation_data: 
                simulation_data[session_id] = {}
            
            simulation_data[session_id][trial] = mot_data
            print(mot_data.shape)
            
    return simulation_data 


# Load simulation for data for given list of sessions
def load_simulation_data(mcs_sessions):
    simulation_data = {}

    for session_id in tqdm.tqdm(mcs_sessions): 
        simulation_results_path = os.path.join(base_dir, session_id, 'OpenSimData','Dynamics')
        for trial in os.listdir(simulation_results_path): 
            
            if trial == "SQT01": # Contains same results as segment-1 
                continue

            mot_file = os.path.join(simulation_results_path, trial, f'kinematics_activations_{trial}_torque_driven.mot')

            if not os.path.isfile(mot_file): 
                print(f"Simulation did not converge: ", session_id,trial)
                continue 
            mot_headers = read_header(mot_file)
            
            if len(mot_headers) == 0:
                print(f"Unable to load headers for file:", session_id,trial)
                continue 
            
            mot_headers = mot_headers
            
            # Remove time from headers
            mot_headers.remove('time')
            
            mot_data = data = storage_to_dataframe(mot_file, headers = mot_headers)
            
            if session_id not in simulation_data: 
                simulation_data[session_id] = {}
            
            simulation_data[session_id][trial] = mot_data
            print(mot_data.shape)
            
    return simulation_data 

In [9]:
# PPE Files containing with MCS Scores
mcs_sessions = ["349e4383-da38-4138-8371-9a5fed63a56a","015b7571-9f0b-4db4-a854-68e57640640d","c613945f-1570-4011-93a4-8c8c6408e2cf","dfda5c67-a512-4ca2-a4b3-6a7e22599732","7562e3c0-dea8-46f8-bc8b-ed9d0f002a77","275561c0-5d50-4675-9df1-733390cd572f","0e10a4e3-a93f-4b4d-9519-d9287d1d74eb","a5e5d4cd-524c-4905-af85-99678e1239c8","dd215900-9827-4ae6-a07d-543b8648b1da","3d1207bf-192b-486a-b509-d11ca90851d7","c28e768f-6e2b-4726-8919-c05b0af61e4a","fb6e8f87-a1cc-48b4-8217-4e8b160602bf","e6b10bbf-4e00-4ac0-aade-68bc1447de3e","d66330dc-7884-4915-9dbb-0520932294c4","0d9e84e9-57a4-4534-aee2-0d0e8d1e7c45","2345d831-6038-412e-84a9-971bc04da597","0a959024-3371-478a-96da-bf17b1da15a9","ef656fe8-27e7-428a-84a9-deb868da053d","c08f1d89-c843-4878-8406-b6f9798a558e","d2020b0e-6d41-4759-87f0-5c158f6ab86a","8dc21218-8338-4fd4-8164-f6f122dc33d9"]

mcs_scores = [4,4,2,3,2,4,3,3,2,3,0,3,4,2,2,3,4,4,3,3,3]
mcs_scores = dict(zip(mcs_sessions,mcs_scores))

simulation_data = load_simulation_data(mcs_sessions)

 10%|▉         | 2/21 [00:00<00:01, 12.49it/s]

(134, 46)
(31, 46)
(133, 46)
(191, 46)
(132, 46)
Simulation did not converge:  c613945f-1570-4011-93a4-8c8c6408e2cf SQT01_segment_3
Simulation did not converge:  c613945f-1570-4011-93a4-8c8c6408e2cf SQT01_segment_1
Simulation did not converge:  c613945f-1570-4011-93a4-8c8c6408e2cf SQT01_segment_2
Simulation did not converge:  dfda5c67-a512-4ca2-a4b3-6a7e22599732 SQT01_segment_3
Simulation did not converge:  dfda5c67-a512-4ca2-a4b3-6a7e22599732 SQT01_segment_1
Simulation did not converge:  dfda5c67-a512-4ca2-a4b3-6a7e22599732 SQT01_segment_2
(156, 46)
(447, 46)
(146, 46)
(156, 46)
(138, 46)


 43%|████▎     | 9/21 [00:00<00:00, 20.56it/s]

(159, 46)
(163, 46)
Simulation did not converge:  0e10a4e3-a93f-4b4d-9519-d9287d1d74eb SQT01_segment_1
(157, 46)
(141, 46)
(162, 46)
(139, 46)
(165, 46)
(603, 46)
(157, 46)
(127, 46)


 62%|██████▏   | 13/21 [00:00<00:00, 23.80it/s]

(232, 46)
(143, 46)
(108, 46)
Simulation did not converge:  c28e768f-6e2b-4726-8919-c05b0af61e4a SQT01_segment_1
(103, 46)
Simulation did not converge:  fb6e8f87-a1cc-48b4-8217-4e8b160602bf SQT01_segment_3
Simulation did not converge:  fb6e8f87-a1cc-48b4-8217-4e8b160602bf SQT01_segment_1
Simulation did not converge:  fb6e8f87-a1cc-48b4-8217-4e8b160602bf SQT01_segment_2
(130, 46)
(225, 46)
(122, 46)
Simulation did not converge:  d66330dc-7884-4915-9dbb-0520932294c4 SQT01_segment_3
(554, 46)
(116, 46)
Simulation did not converge:  0d9e84e9-57a4-4534-aee2-0d0e8d1e7c45 SQT01_segment_3
(422, 46)
Simulation did not converge:  0d9e84e9-57a4-4534-aee2-0d0e8d1e7c45 SQT01_segment_2


100%|██████████| 21/21 [00:00<00:00, 25.08it/s]

(171, 46)
(229, 46)
Simulation did not converge:  2345d831-6038-412e-84a9-971bc04da597 SQT01_segment_2
Simulation did not converge:  0a959024-3371-478a-96da-bf17b1da15a9 SQT01_segment_3
Simulation did not converge:  0a959024-3371-478a-96da-bf17b1da15a9 SQT01_segment_1
Simulation did not converge:  0a959024-3371-478a-96da-bf17b1da15a9 SQT01_segment_2
Simulation did not converge:  ef656fe8-27e7-428a-84a9-deb868da053d SQT01_segment_3
Simulation did not converge:  ef656fe8-27e7-428a-84a9-deb868da053d SQT01_segment_1
Simulation did not converge:  ef656fe8-27e7-428a-84a9-deb868da053d SQT01_segment_2
(122, 46)
Simulation did not converge:  c08f1d89-c843-4878-8406-b6f9798a558e SQT01_segment_1
(124, 46)
(197, 46)
(241, 46)
(150, 46)
(112, 46)
Simulation did not converge:  8dc21218-8338-4fd4-8164-f6f122dc33d9 SQT01_segment_1
(115, 46)


In [10]:
import os
import numpy as np
import pandas as pd
import tqdm
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_box_plots(simulation_data, feature_names, features_per_fig=12):
    num_features = len(feature_names)
    num_figures = int(np.ceil(num_features / features_per_fig))  # Divide features into multiple figures

    # Identify the minimum time length across all trials
    min_time_length = np.inf
    for session_id, trials in simulation_data.items():
        for trial_id, df in trials.items():
            min_time_length = min(min_time_length, len(df.index))  # Find the shortest time length

    print(f"Minimum time length across all trials: {min_time_length}")

    # Iterate through and create multiple figures
    for fig_idx in range(num_figures):
        start_idx = fig_idx * features_per_fig
        end_idx = min(start_idx + features_per_fig, num_features)
        features_to_plot = feature_names[start_idx:end_idx]

        # Create subplots for the current figure
        rows = int(np.ceil(len(features_to_plot) / 3))
        fig = make_subplots(rows=rows, cols=3, subplot_titles=features_to_plot, vertical_spacing=0.1)

        for i, feature in enumerate(features_to_plot):
            row = i // 3 + 1
            col = i % 3 + 1

            feature_data = []
            time_points = None
            for session_id, trials in simulation_data.items():
                for trial_id, df in trials.items():
                    # Collect feature data for all subjects, trimming to min_time_length
                    feature_data.append(df[feature].values[:min_time_length])
                    time_points = df.index.values[:min_time_length]

            # Convert the list of arrays into a 2D numpy array (T x num_trials)
            feature_data = np.array(feature_data).T  # shape: T x num_trials

            # Create box plot for each time point
            for t, time_point in enumerate(time_points):
                fig.add_trace(go.Box(
                    y=feature_data[t],
                    name=str(time_point),
                    boxmean='sd',
                    marker_color='gray',  # Monochrome color
                    boxpoints = False # Hide outliers
                ), row=row, col=col)

        # Update layout for the figure
        fig.update_layout(
            height=300 * rows,  # Dynamically adjust height based on the number of rows
            width=1000,
            title_text=f"Mean and Standard Deviation of Features Across Subjects (Features {start_idx+1}-{end_idx})",
            showlegend=False  # No legend
        )

        # Adjust x-axis labels
        fig.update_xaxes(tickvals=time_points, ticktext=[str(t) for t in time_points])

        # Show the current figure
        fig.show()

In [ ]:
subject_trials = list(simulation_data.keys())
trial_data = simulation_data[subject_trials[0]]  # Get first session for example
feature_names = trial_data[list(trial_data.keys())[0]].columns[1:]  # Assuming 'time' is the first column

# Call the box plot function
plot_box_plots(simulation_data, feature_names)

In [11]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_simulation_data(trial_mot_data, mcs_scores): 
        
    # Create 4x4 subplots (we'll only use 14 of them)
    fig = make_subplots(rows=5, cols=3, subplot_titles=(
        'Trunk Obliquity', 'Trunk Tilt', 'Trunk Rotation',
        'Pelvic Obliquity', 'Pelvic Tilt', 'Pelvic Rotation',
        'Left Hip Ab/Adduction', 'Left Hip Flexion/Extension', 'Left Hip Rotation',
        'Right Hip Ab/Adduction', 'Right Hip Flexion/Extension', 'Right Hip Rotation',
        'Left Knee Flexion/Extension', 'Right Knee Flexion/Extension',
        'Ankle Dorsi/Plantar'
    ))

    # Define the mapping of DataFrame columns to plot titles
    plot_mapping = {
        'Trunk Obliquity': 'lumbar_bending',
        'Trunk Tilt': 'lumbar_extension',
        'Trunk Rotation': 'lumbar_rotation',
        'Pelvic Obliquity': 'pelvis_list',
        'Pelvic Tilt': 'pelvis_tilt',
        'Pelvic Rotation': 'pelvis_rotation',
        'Left Hip Ab/Adduction': 'hip_adduction_l',
        'Left Hip Flexion/Extension': 'hip_flexion_l',
        'Left Hip Rotation': 'hip_rotation_l',
        'Right Hip Ab/Adduction': 'hip_adduction_r',
        'Right Hip Flexion/Extension': 'hip_flexion_r',
        'Right Hip Rotation': 'hip_rotation_r',
        'Knee Varus/Valgus': ['knee_angle_l', 'knee_angle_r'],  # Assuming this corresponds to knee angle
        'Knee Flexion/Extension': ['knee_angle_l', 'knee_angle_r'],
        'Knee Rotation': ['knee_angle_l', 'knee_angle_r'],  # Assuming knee rotation is not directly available
        'Ankle Dorsi/Plantar': ['ankle_angle_l', 'ankle_angle_r'],
        'Foot Progress Angles': ['subtalar_angle_l', 'subtalar_angle_r']  # Assuming this corresponds to foot progress
    }

    # Colors for left and right sides
    colors = {'left': 'blue', 'right': 'red'}

    # Create each subplot
    for i, (title, columns) in enumerate(plot_mapping.items()):
        row = i // 3 + 1
        col = i % 3 + 1

        for i,trial in enumerate(trial_mot_data):
            df = trial_mot_data[trial]
            
            if isinstance(columns, list):
                print(columns)
                # Plot both left and right
                fig.add_trace(go.Scatter(x=df['time']-df['time'][0], y=df[columns[0]], name=f'{i} {title} (Left)', line=dict(color=colors['left'])), row=row, col=col)
                fig.add_trace(go.Scatter(x=df['time']-df['time'][0], y=df[columns[1]], name=f'{i} {title} (Right)', line=dict(color=colors['right'])), row=row, col=col)
            else:
                # Plot single trace
                fig.add_trace(go.Scatter(x=df['time']-df['time'][0], y=df[columns], name=title), row=row, col=col)
        
        # Update y-axis label
        fig.update_yaxes(title_text='deg', row=row, col=col)

    # Update x-axis label for the bottom row
    for col in range(1, 4):
        fig.update_xaxes(title_text='%SQT Cycle', row=4, col=col)

    # Update layout
    fig.update_layout(height=1200, width=1000, title_text="Biomechanical Model Kinematics")

    # Show the figure
    fig.show()

In [8]:
# for session_id in tqdm.tqdm(mcs_sessions):
#     if session_id in simulation_data and session_id in mcs_scores and 0 < mcs_scores[session_id] < 5: 
#         plot_simulation_data(simulation_data[session_id], mcs_scores[session_id])
    

In [65]:
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_simulation_data(session_id,trial_mot_data, mcs_score,subject_id): 
        
    # Create 4x4 subplots (we'll only use 14 of them)
    fig = make_subplots(rows=6, cols=3, subplot_titles=(
        'Trunk Obliquity', 'Trunk Tilt', 'Trunk Rotation',
        'Pelvic Obliquity', 'Pelvic Tilt', 'Pelvic Rotation',
        'Left Hip Ab/Adduction', 'Left Hip Flexion/Extension', 'Left Hip Rotation',
        'Right Hip Ab/Adduction', 'Right Hip Flexion/Extension', 'Right Hip Rotation',
        'Left Knee Flexion/Extension', 'Right Knee Flexion/Extension', '',
        'Left Ankle Dorsi/Plantar', 'Right Ankle Dorsi/Plantar', ''
    ))

    # Define the mapping of DataFrame columns to plot titles
    plot_mapping = {
        'Trunk Obliquity': 'lumbar_bending',
        'Trunk Tilt': 'lumbar_extension',
        'Trunk Rotation': 'lumbar_rotation',
        'Pelvic Obliquity': 'pelvis_list',
        'Pelvic Tilt': 'pelvis_tilt',
        'Pelvic Rotation': 'pelvis_rotation',
        'Left Hip Ab/Adduction': 'hip_adduction_l',
        'Left Hip Flexion/Extension': 'hip_flexion_l',
        'Left Hip Rotation': 'hip_rotation_l',
        'Right Hip Ab/Adduction': 'hip_adduction_r',
        'Right Hip Flexion/Extension': 'hip_flexion_r',
        'Right Hip Rotation': 'hip_rotation_r',
        'Left Knee Flexion/Extension': 'knee_angle_l',
        'Right Knee Flexion/Extension': 'knee_angle_r',
        'Knee Flexion/Extension': '',
        'Left Ankle Dorsi/Plantar': 'ankle_angle_l',
        'Right Ankle Dorsi/Plantar': 'ankle_angle_r', 
        'Ankle Dorsi/Plantar': '',
    }

    # Colors for left and right sides
    colors = {'left': 'blue', 'right': 'red'}

    # Create each subplot
    for i, (title, columns) in enumerate(plot_mapping.items()):
        row = i // 3 + 1
        col = i % 3 + 1

        if columns == '': 
            continue  

        for trial_id,trial in enumerate(trial_mot_data):
            df = trial_mot_data[trial]
            
            # Plot single trace
            fig.add_trace(go.Scatter(x=np.linspace(0,1,num=101), y=np.array(df[columns]), name=f'{trial_id} {title}'), row=row, col=col)
    
        # Update y-axis label
        fig.update_yaxes(title_text='deg', row=row, col=col)

    # Update x-axis label for the bottom row
    for col in range(1, 6):
        for row in range(1,6):
            fig.update_xaxes(title_text='% SQT Cycle (Seconds)', row=row, col=col)

    # Update layout
    fig.update_layout(height=1500, width=1000,
                        showlegend=False,  title_x=0.5,
                        title_text=f"OpenCap Simulation Kinematics <br> Subject:{subject_id} MCS Score:{mcs_score}",
                        font_family="Times New Roman",
                        font_color="black",
                        title_font_family="Times New Roman",
                        title_font_color="black")

    # Show the figure
    fig.show()
    
    return fig

In [66]:
# Crop segment 1
def crop_segmemt_1(mot_dict): 
    
    min_time = min([ mot_dict[k].shape[0] for k in mot_dict ])
    
    if 'SQT01_segment_1' in mot_dict:
        mot_dict['SQT01_segment_1'] = mot_dict['SQT01_segment_1'].tail(min_time)
    
    return mot_dict


In [67]:
# PPE Files containing with MCS Scores
mcs_sessions = ["349e4383-da38-4138-8371-9a5fed63a56a","015b7571-9f0b-4db4-a854-68e57640640d","c613945f-1570-4011-93a4-8c8c6408e2cf","dfda5c67-a512-4ca2-a4b3-6a7e22599732","7562e3c0-dea8-46f8-bc8b-ed9d0f002a77","275561c0-5d50-4675-9df1-733390cd572f","0e10a4e3-a93f-4b4d-9519-d9287d1d74eb","a5e5d4cd-524c-4905-af85-99678e1239c8","dd215900-9827-4ae6-a07d-543b8648b1da","3d1207bf-192b-486a-b509-d11ca90851d7","c28e768f-6e2b-4726-8919-c05b0af61e4a","fb6e8f87-a1cc-48b4-8217-4e8b160602bf","e6b10bbf-4e00-4ac0-aade-68bc1447de3e","d66330dc-7884-4915-9dbb-0520932294c4","0d9e84e9-57a4-4534-aee2-0d0e8d1e7c45","2345d831-6038-412e-84a9-971bc04da597","0a959024-3371-478a-96da-bf17b1da15a9","ef656fe8-27e7-428a-84a9-deb868da053d","c08f1d89-c843-4878-8406-b6f9798a558e","d2020b0e-6d41-4759-87f0-5c158f6ab86a","8dc21218-8338-4fd4-8164-f6f122dc33d9"]

mcs_scores = [4,4,2,3,2,4,3,3,2,3,0,3,4,2,2,3,4,4,3,3,3]
mcs_scores = dict(zip(mcs_sessions,mcs_scores))

PPE_Subjects = ["PPE09182201","PPE09182202","PPE09182203","PPE09182204","PPE09182205","PPE09182206","PPE09182207","PPE09182208","PPE09182209","PPE091822010","PPE09182211","PPE09182212","PPE09182213","PPE09182214","PPE09182215","PPE09182216","PPE09182217","PPE09182218","PPE09182219","PPE09182220","PPE09182221"]
PPE_Subjects = dict(zip(mcs_sessions,PPE_Subjects))

simulation_data = load_simulation_data(mcs_sessions)

 10%|▉         | 2/21 [00:00<00:01, 12.99it/s]

(134, 46)
(31, 46)
(133, 46)
(191, 46)
(132, 46)
Simulation did not converge:  c613945f-1570-4011-93a4-8c8c6408e2cf SQT01_segment_3
Simulation did not converge:  c613945f-1570-4011-93a4-8c8c6408e2cf SQT01_segment_1
Simulation did not converge:  c613945f-1570-4011-93a4-8c8c6408e2cf SQT01_segment_2
Simulation did not converge:  dfda5c67-a512-4ca2-a4b3-6a7e22599732 SQT01_segment_3
Simulation did not converge:  dfda5c67-a512-4ca2-a4b3-6a7e22599732 SQT01_segment_1
Simulation did not converge:  dfda5c67-a512-4ca2-a4b3-6a7e22599732 SQT01_segment_2
(156, 46)
(447, 46)
(146, 46)
(156, 46)


 29%|██▊       | 6/21 [00:00<00:00, 21.19it/s]

(138, 46)
(159, 46)
(163, 46)
Simulation did not converge:  0e10a4e3-a93f-4b4d-9519-d9287d1d74eb SQT01_segment_1
(157, 46)
(141, 46)
(162, 46)
(139, 46)


 43%|████▎     | 9/21 [00:00<00:00, 14.18it/s]

(165, 46)
(603, 46)
(157, 46)
(127, 46)
(232, 46)
(143, 46)
(108, 46)
Simulation did not converge:  c28e768f-6e2b-4726-8919-c05b0af61e4a SQT01_segment_1


 76%|███████▌  | 16/21 [00:00<00:00, 19.52it/s]

(103, 46)
Simulation did not converge:  fb6e8f87-a1cc-48b4-8217-4e8b160602bf SQT01_segment_3
Simulation did not converge:  fb6e8f87-a1cc-48b4-8217-4e8b160602bf SQT01_segment_1
Simulation did not converge:  fb6e8f87-a1cc-48b4-8217-4e8b160602bf SQT01_segment_2
(130, 46)
(225, 46)
(122, 46)
Simulation did not converge:  d66330dc-7884-4915-9dbb-0520932294c4 SQT01_segment_3
(554, 46)
(116, 46)
Simulation did not converge:  0d9e84e9-57a4-4534-aee2-0d0e8d1e7c45 SQT01_segment_3
(422, 46)
Simulation did not converge:  0d9e84e9-57a4-4534-aee2-0d0e8d1e7c45 SQT01_segment_2
(171, 46)
(229, 46)
Simulation did not converge:  2345d831-6038-412e-84a9-971bc04da597 SQT01_segment_2
Simulation did not converge:  0a959024-3371-478a-96da-bf17b1da15a9 SQT01_segment_3
Simulation did not converge:  0a959024-3371-478a-96da-bf17b1da15a9 SQT01_segment_1
Simulation did not converge:  0a959024-3371-478a-96da-bf17b1da15a9 SQT01_segment_2
Simulation did not converge:  ef656fe8-27e7-428a-84a9-deb868da053d SQT01_segment

100%|██████████| 21/21 [00:01<00:00, 20.93it/s]

(124, 46)
(197, 46)
(241, 46)
(150, 46)
(112, 46)
Simulation did not converge:  8dc21218-8338-4fd4-8164-f6f122dc33d9 SQT01_segment_1
(115, 46)


In [108]:
from scipy.interpolate import CubicSpline
def convert2splines(simulation_data): 

    time_normalized_simulation_data = {}
    for trial in simulation_data: 
        
        T = simulation_data[trial].shape
        spline_data = []
        time_data = np.array(simulation_data[trial]['time'])
        for header_ind, (header, series) in enumerate(simulation_data[trial].items()):
            
            spline = CubicSpline(time_data, np.array(series))

            spline_input = np.linspace(time_data[0],time_data[-1],101)
            spline_data.append(spline(spline_input))
            
        spline_data = np.array(spline_data)
        out = pd.DataFrame(data=spline_data.T, columns=simulation_data[trial].columns)
        
        time_normalized_simulation_data[trial] = out
    
    return time_normalized_simulation_data
            

In [ ]:


skip_sessions = [mcs_sessions[0], ]
save_locations = {}
os.makedirs("pdfs",exist_ok=True)

time_normalized_simulation_data = {} 

for session_id in tqdm.tqdm(mcs_sessions):
    # Check if all the details that have to be plotted exist
    if session_id in simulation_data \
        and session_id in PPE_Subjects\
        and  session_id in mcs_scores\
        and 0 < mcs_scores[session_id] < 5: 
        
        if session_id in skip_sessions: 
            continue 
        
        simulation_data[session_id] = crop_segmemt_1(simulation_data[session_id])
        
        time_normalized_simulation_data[session_id] = convert2splines(simulation_data[session_id])
        
        
        fig = plot_simulation_data(session_id, \
            time_normalized_simulation_data[session_id], \
            mcs_scores[session_id],\
            PPE_Subjects[session_id])

        plotly.io.write_image(fig, f'pdfs/{session_id}_mcs_{mcs_scores[session_id]}.pdf', format='pdf')
        

In [110]:
plot_mapping = {
        'Trunk Obliquity': 'lumbar_bending',
        'Trunk Tilt': 'lumbar_extension',
        'Trunk Rotation': 'lumbar_rotation',
        'Pelvic Obliquity': 'pelvis_list',
        'Pelvic Tilt': 'pelvis_tilt',
        'Pelvic Rotation': 'pelvis_rotation',
        'Left Hip Ab/Adduction': 'hip_adduction_l',
        'Left Hip Flexion/Extension': 'hip_flexion_l',
        'Left Hip Rotation': 'hip_rotation_l',
        'Right Hip Ab/Adduction': 'hip_adduction_r',
        'Right Hip Flexion/Extension': 'hip_flexion_r',
        'Right Hip Rotation': 'hip_rotation_r',
        'Left Knee Flexion/Extension': 'knee_angle_l',
        'Right Knee Flexion/Extension': 'knee_angle_r',
        'Left Ankle Dorsi/Plantar': 'ankle_angle_l',
        'Right Ankle Dorsi/Plantar': 'ankle_angle_r', 
    }
list(plot_mapping.values())

['lumbar_bending',
 'lumbar_extension',
 'lumbar_rotation',
 'pelvis_list',
 'pelvis_tilt',
 'pelvis_rotation',
 'hip_adduction_l',
 'hip_flexion_l',
 'hip_rotation_l',
 'hip_adduction_r',
 'hip_flexion_r',
 'hip_rotation_r',
 'knee_angle_l',
 'knee_angle_r',
 'ankle_angle_l',
 'ankle_angle_r']

In [1]:
# Plot mean and std using time-normalized data
headers = ['lumbar_bending', 'lumbar_extension', 'lumbar_rotation', 'pelvis_list', 'pelvis_tilt', 'pelvis_rotation', 'hip_adduction_l', 'hip_flexion_l', 'hip_rotation_l', 'hip_adduction_r', 'hip_flexion_r', 'hip_rotation_r', 'knee_angle_l', 'knee_angle_r', 'ankle_angle_l', 'ankle_angle_r']

mean_data = np.zeros((101,len(headers)))
std_data = np.zeros((101,len(headers)))
total_trials = 0
for session_id in time_normalized_simulation_data: 
    for trail in time_normalized_simulation_data[session_id]: 
        data = time_normalized_simulation_data[session_id][trail][headers].to_numpy()
        mean_data += data
        std_data += data**2
        total_trials += 1

mean_data /= total_trials
std_data = np.sqrt(std_data/total_trials - mean_data) 

mean_df = pd.DataFrame(data=mean_data, columns=headers)
std_df = pd.DataFrame(data=mean_data, columns=headers)

# Create 4x4 subplots (we'll only use 14 of them)
fig = make_subplots(rows=6, cols=3, subplot_titles=(
    'Trunk Obliquity', 'Trunk Tilt', 'Trunk Rotation',
    'Pelvic Obliquity', 'Pelvic Tilt', 'Pelvic Rotation',
    'Left Hip Ab/Adduction', 'Left Hip Flexion/Extension', 'Left Hip Rotation',
    'Right Hip Ab/Adduction', 'Right Hip Flexion/Extension', 'Right Hip Rotation',
    'Left Knee Flexion/Extension', 'Right Knee Flexion/Extension', '',
    'Left Ankle Dorsi/Plantar', 'Right Ankle Dorsi/Plantar', ''
))

# Define the mapping of DataFrame columns to plot titles
plot_mapping = {
    'Trunk Obliquity': 'lumbar_bending',
    'Trunk Tilt': 'lumbar_extension',
    'Trunk Rotation': 'lumbar_rotation',
    'Pelvic Obliquity': 'pelvis_list',
    'Pelvic Tilt': 'pelvis_tilt',
    'Pelvic Rotation': 'pelvis_rotation',
    'Left Hip Ab/Adduction': 'hip_adduction_l',
    'Left Hip Flexion/Extension': 'hip_flexion_l',
    'Left Hip Rotation': 'hip_rotation_l',
    'Right Hip Ab/Adduction': 'hip_adduction_r',
    'Right Hip Flexion/Extension': 'hip_flexion_r',
    'Right Hip Rotation': 'hip_rotation_r',
    'Left Knee Flexion/Extension': 'knee_angle_l',
    'Right Knee Flexion/Extension': 'knee_angle_r',
    'Knee Flexion/Extension': '',
    'Left Ankle Dorsi/Plantar': 'ankle_angle_l',
    'Right Ankle Dorsi/Plantar': 'ankle_angle_r', 
    'Ankle Dorsi/Plantar': '',
}

# Colors for left and right sides
colors = {'left': 'blue', 'right': 'red'}



# Create each subplot
for i, (title, columns) in enumerate(plot_mapping.items()):
    row = i // 3 + 1
    col = i % 3 + 1

    if columns == '': 
        continue  
    
    
    
    # Plot single trace
    
    fig.add_trace(go.Scatter(x=np.linspace(0,1,num=101), y=np.array(mean_df[columns]), showlegend=False, name=f'{title}'), row=row, col=col)
    x = np.linspace(0,1,num=101)
    fig.add_trace(go.Scatter(x=list(x) + list(x)[::-1],   y=list(np.array(mean_df[columns]) + np.array(std_df[columns])) + list(np.array(mean_df[columns]) - np.array(std_df[columns]))[::-1] , mode='lines', line=dict(width=0), name=f'{title} Upper Bound', showlegend=False, fill='toself',hoverinfo="skip",), row=row, col=col)

    # Update y-axis label
    fig.update_yaxes(title_text='deg', row=row, col=col)

# Update x-axis label for the bottom row
for col in range(1, 6):
    for row in range(1,6):
        fig.update_xaxes(title_text='% SQT Cycle (Seconds)', row=row, col=col)

# Update layout
fig.update_layout(height=1500, width=1000,
                    showlegend=False,  title_x=0.5,
                    title_text=f"All subjects Kinematics",
                    font_family="Times New Roman",
                    font_color="black",
                    title_font_family="Times New Roman",
                    title_font_color="black")

# Show the figure
fig.show()

plotly.io.write_image(fig, f'pdfs/all_subject_kinematics.pdf', format='pdf')



NameError: name 'np' is not defined

In [13]:
!pip install pypdf

In [107]:
from pypdf import PdfMerger
import glob

pdfs = os.listdir("pdfs")
pdfs.remove("all_subject_kinematics.pdf")
pdfs.insert(0,"all_subject_kinematics.pdf")
merger = PdfMerger()

for pdf in pdfs:
    merger.append(os.path.join("pdfs",pdf))

merger.write("MCS-Subjects-SQT-Kinematics.pdf")
merger.close()

/home/ubuntu/.conda/envs/mdm/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning:

PdfMerger is deprecated and will be removed in pypdf 5.0.0. Use PdfWriter instead.



In [101]:
pdfs = os.listdir("pdfs")

In [102]:
pdfs

['0e10a4e3-a93f-4b4d-9519-d9287d1d74eb_mcs_3.pdf',
 '3d1207bf-192b-486a-b509-d11ca90851d7_mcs_3.pdf',
 '00000all_subject_kinematics.pdf',
 'd66330dc-7884-4915-9dbb-0520932294c4_mcs_2.pdf',
 'dd215900-9827-4ae6-a07d-543b8648b1da_mcs_2.pdf',
 '8dc21218-8338-4fd4-8164-f6f122dc33d9_mcs_3.pdf',
 '275561c0-5d50-4675-9df1-733390cd572f_mcs_4.pdf',
 '0d9e84e9-57a4-4534-aee2-0d0e8d1e7c45_mcs_2.pdf',
 '015b7571-9f0b-4db4-a854-68e57640640d_mcs_4.pdf',
 'a5e5d4cd-524c-4905-af85-99678e1239c8_mcs_3.pdf',
 'e6b10bbf-4e00-4ac0-aade-68bc1447de3e_mcs_4.pdf',
 '7562e3c0-dea8-46f8-bc8b-ed9d0f002a77_mcs_2.pdf',
 'd2020b0e-6d41-4759-87f0-5c158f6ab86a_mcs_3.pdf',
 'c08f1d89-c843-4878-8406-b6f9798a558e_mcs_3.pdf',
 '2345d831-6038-412e-84a9-971bc04da597_mcs_3.pdf']